In [19]:
import pandas as pd
import requests
import bs4 as bs
import urllib.request

from general_functions import get_file_contents
from general_functions import load_xml_to_dataframe


filename = 'ServiceKey 국토교통부 아파트매매 실거래.txt'
api_key = get_file_contents(filename)

In [20]:
def load_xml_to_dataframe(response_xml, keyword):
    import urllib.request

    import bs4 as bs
    import pandas as pd

    soup = bs.BeautifulSoup(response_xml, 'xml')

    rows = soup.find_all(keyword)
    columns = rows[0].find_all()

    rowList = []
    nameList = []
    columnList = []

    rowsLen = len(rows)
    columnsLen = len(columns)

    for i in range(0, rowsLen):
        columns = rows[i].find_all()

        for j in range(0, columnsLen):
            if i == 0:
                nameList.append(columns[j].name)  # header
            eachColumn = columns[j].text  # value
            columnList.append(eachColumn)
        rowList.append(columnList)
        columnList = []    # 다음 row의 값을 넣기 위해 비워준다

    result = pd.DataFrame(rowList, columns=nameList)
    return result

In [21]:
df = pd.DataFrame()
for ymd in ['202101', '202204', '202205']:
        params = {'serviceKey': api_key, 'LAWD_CD': '11110', 'DEAL_YMD': ymd}
        d = load_xml_to_dataframe(requests.get(url,
                                                 params=params
                                                 ).text, 'item')
        df =df.append(d, ignore_index = True)
        

response = requests.get(url, params=params).text


In [22]:
soup = bs.BeautifulSoup(response,'xml')

import xmltodict
xmltodict.parse(response)

In [23]:
for col in ['거래금액', '건축년도', '년', '층']:
    df[col] = pd.to_numeric(df[col].str.replace(",", ""))

In [24]:
import numpy as np
pd.pivot_table(data=df, index='법정동', columns='거래유형', values='거래금액', aggfunc=sum, fill_value=0)

거래유형,,중개거래,직거래
법정동,,,
견지동,127000,0,0
교북동,114000,114000,0
내수동,710000,185000,200100
명륜1가,63000,18500,0
명륜2가,430000,87000,78000
무악동,1231730,517490,0
사직동,150000,0,0
숭인동,400550,305500,10000
연건동,0,24000,18000


In [25]:
import pydeck as pdk
import streamlit as st

In [26]:
import os
from dotenv import load_dotenv

load_dotenv(verbose=True)
token = os.getenv('MAPBOX_API_KEY')

In [27]:
result = pd.pivot_table(data=df, index='법정동', values='거래금액', aggfunc='sum', fill_value=0).sort_values('거래금액', ascending=False)
result

,거래금액
법정동,
무악동,1749220
창신동,1131200
내수동,1095100
평동,865000
숭인동,716050
명륜2가,595000
홍파동,350000
평창동,316900
교북동,228000


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [29]:
df

,거래금액,거래유형,건축년도,년,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,130000,,2000,2021,청운동,청운현대,1,14,129.76,,56-45,11110,2,,
1,150000,,2008,2021,사직동,광화문스페이스본(106동),1,7,144.52,,9-1,11110,6,,
2,175000,,2004,2021,내수동,경희궁의아침2단지,1,15,174.55,,71,11110,4,,
3,175000,,2004,2021,내수동,경희궁의아침2단지,1,15,174.55,,71,11110,4,21.04.08,O
4,180000,,2004,2021,내수동,경희궁의아침3단지,1,17,123.13,,72,11110,13,21.04.19,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,164000,중개거래,2008,2022,무악동,인왕산아이파크,4,2,114.931,서울 종로구,60,11110,16,,
76,112490,중개거래,2000,2022,무악동,현대,4,9,60,서울 종로구,82,11110,13,,
77,241000,중개거래,2008,2022,무악동,인왕산아이파크,4,26,157.289,서울 종로구,60,11110,15,,
78,200100,직거래,2004,2022,내수동,경희궁의아침3단지,5,10,174.55,,72,11110,4,,
